# Existe relação diretamente o IBC-Br e o PIB?

# O que é o PIB?

O Produto Interno Bruto (PIB) representa a soma total de todos os bens e serviços finais gerados dentro de um país, estado ou cidade, tipicamente calculados anualmente. Cada nação avalia seu PIB em sua própria moeda. Por exemplo, o PIB do Brasil alcançou R$ 10,1 trilhões em 2022, e no terceiro trimestre de 2023, este valor atingiu R$ 2.741,0 bilhões.

Para calcular o PIB, apenas os bens e serviços finais são contabilizados, evitando assim a contagem dupla. Por exemplo, se um país produzir trigo no valor de R$ 100, farinha de trigo por R$ 200 e pão por R$ 300, o PIB será contado como R$ 300. Isso acontece porque o valor do trigo e da farinha já está incluído no preço final do pão.

Importante destacar, o PIB mede esses bens e serviços finais a preços de consumidor, o que inclui os impostos embutidos nos produtos comercializados.

É um equívoco comum confundir o PIB com a riqueza total de um país. O PIB não reflete o 'estoque' de riqueza existente, mas sim um 'fluxo' de novos bens e serviços produzidos em um determinado período. Portanto, se em um ano um país não produzir nada, seu PIB será zero.

# O que é o IBC-Br?

O Índice de Atividade Econômica do Banco Central (IBC-Br) é uma métrica desenvolvida pelo Banco Central do Brasil (BACEN) para monitorar e antecipar tendências na economia brasileira. Ele serve como um indicador de desempenho econômico, refletindo a atividade em diversos setores da economia em um determinado período, geralmente mensal.

O IBC-Br é calculado com base em uma variedade de indicadores, incluindo os níveis de produção industrial, vendas no varejo e serviços, entre outros. Ao contrário do Produto Interno Bruto (PIB), que é calculado trimestralmente, o IBC-Br fornece uma visão mais frequente e atualizada da economia.

Este índice é particularmente útil para os formuladores de políticas econômicas e investidores, pois oferece uma visão preliminar da direção em que a economia está se movendo, antes mesmo da divulgação oficial do PIB. Isso permite reações e ajustes mais rápidos às tendências econômicas.

É importante destacar que, embora o IBC-Br seja um indicador abrangente, ele não mede todos os aspectos da economia. Por exemplo, ele não inclui dados sobre a economia informal. Além disso, assim como o PIB, o IBC-Br não deve ser confundido com um indicador de riqueza ou bem-estar, mas sim como um reflexo do nível de atividade econômica.

---

Após compreendermos os conceitos e os fatores determinantes que influenciam cada um dos indicadores econômicos, uma questão relevante emerge: existe uma forte correlação entre o IBC-Br, considerado uma prévia do PIB, e o próprio PIB? Para responder a esta pergunta, vamos usar o Python como ferramenta de coleta, tratamento, análise e modelagem dos dados para verificar esse argumento.

## Importando bibliotecas

In [1]:
# Pandas: Data manipulation and analysis library.
# https://pandas.pydata.org
try:
    import pandas as pd
except ModuleNotFoundError:
    print('Pandas is not installed, installing now...')
    %pip install pandas
finally:
    import pandas as pd

In [2]:
# Matplotlib: Data visualization library
# https://matplotlib.org
try:
    import matplotlib.pyplot as plt
except ModuleNotFoundError:
    print('Matplotlib is not installed, installing now...')
    %pip install matplotlib
finally:
    import matplotlib.pyplot as plt

In [3]:
# Plotly: Interactive data visualization library
# https://plotly.com
try:
    import plotly.express as px
except ModuleNotFoundError:
    print('Plotly is not installed, installing now...')
    %pip install plotly
finally:
    import plotly.express as px

In [4]:
# Sidrapy: Brazilian IBGE data extraction library
# https://sidrapy.readthedocs.io/pt-br/latest
try:
    import sidrapy as sd
except ModuleNotFoundError:
    print('Sidrapy is not installed, installing now...')
    %pip install sidrapy
finally:
    import sidrapy as sd

In [5]:
# Configuring Matplotlib to display graphics inline.
%matplotlib inline

In [6]:
# Mudando o motor de plotagem padrão do Pandas para o Plotly.
pd.options.plotting.backend = 'plotly'

In [7]:
# Para obter a série histórica do IBC-Br vamos utilizar a API de Dados Abertos do Banco Central do Brasil. 
ibc_br = pd.read_json('https://api.bcb.gov.br/dados/serie/bcdata.sgs.24363/dados?formato=json')
# Ajustando o formato da data e definindo-a como índice do DataFrame.
ibc_br['data'] = pd.to_datetime(ibc_br['data'], dayfirst=True, format='%d/%m/%Y')
ibc_br.set_index('data', inplace=True)

In [8]:
# Visualizando a série histórica do IBC-Br.
fig = ibc_br.plot(
    title='Índice de Atividade Econômica do Banco Central (IBC-Br) - com ajuste sazonal')
# Configurando o layout do gráfico.
fig.update_layout(xaxis_title='',
                  yaxis_title='Índice',
                  showlegend=False,
                  template='plotly_white')

## Disclaimer

O storytelling empregado neste estudo foi projetada com o intuito de tornar os dados apresentados e o método de elaboração compreensível para qualquer pessoa, independentemente de sua familiaridade com o assunto. Se você já possui conhecimento técnico em Python e/ou Ciência de Dados, sinta-se à vontade para ignorar quaisquer comentários que possam parecer redundantes ou enfadonhos.